In [1]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import create_retriever_tool
from langchain.agents import create_agent
from langchain_core.runnables import Runnable
import re
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

def normaliz_cleanir_text(txt):
    text = txt.lower()
    text = re.sub(r"\s+", " ", text)   # свертка пробелов
    text = re.sub(r"\n{2,}", r"\n\n", text)
    d = text.encode('utf-8', 'ignore').decode('utf-8').strip()
    return d

# Функция для форматирования документов
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


class TimedRetriever(Runnable):
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, input, config=None):
        result = self.retriever.invoke(input, config=config)
        print("Выполняется поиск...")
        print('.'*30)
        return result


load_dotenv('my.env')
MODEL = os.getenv("MODEL")
API_KEY = os.getenv("API_KEY")
API_BASE = os.getenv("API_BASE")

llm = ChatOpenAI(
    model=MODEL,
    openai_api_key=API_KEY,
    openai_api_base=API_BASE,
    temperature=0.05
)


with open('docs/Остров сокровищ.txt') as f:
    txt = f.read()

txt = normaliz_cleanir_text(txt)

doc = Document(page_content=txt)

splitters = [
                RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30),
                RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50),
                RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80),
                RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0),
]
chunks = []
for splitter in splitters:
    for chunk_text in splitter.split_text(doc.page_content):
        chunks.append(Document(page_content=chunk_text, metadata={}))

print("Всего чанков: ", len(chunks))
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})



retriever = TimedRetriever(retriever)

# Промпт для RAG
prompt = ChatPromptTemplate.from_template("Ответь на вопрос, без воды, используя только следующий контекст:\nКонтекст: {context}\nВопрос: {question}\nОтвет:")


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

Всего чанков:  2944


In [3]:
q1 = "Что искали на острове пираты?"
print(q1)
response = chain.invoke(q1)
print(response.content)

Что искали на острове пираты?
Выполняется поиск...
..............................
Сокровища Флинта.


In [4]:
q2 = "Продолжи полностью текст песни: пятнадцать человек на..."
print(q2)
response = chain.invoke(q2)
print(response.content)


Продолжи полностью текст песни: пятнадцать человек на...
Выполняется поиск...
..............................
Пятнадцать человек на сундук мертвеца,
Йо-хо-хо, и бутылка рому!
Пятнадцать человек высадились на берег,
Йо-хо-хо, и бутылка рому!

Пятнадцать человек их было вначале,
Йо-хо-хо, и бутылка рому!
Но теперь остался только я,
Йо-хо-хо, и бутылка рому!

Я смотрю на море, оно так сине,
Йо-хо-хо, и бутылка рому!
И думаю о тех, кто не вернулся домой,
Йо-хо-хо, и бутылка рому!

Они потонули в пучине морской,
Йо-хо-хо, и бутылка рому!
И теперь их кости лежат на дне,
Йо-хо-хо, и бутылка рому!

Но я живу, и пью свой ром,
Йо-хо-хо, и бутылка рому!
И пою эту песню о мертвецах,
Йо-хо-хо, и бутылка рому!


In [5]:
q3 = "Кто жил на острове, прежде чем туда приплыли герои книги?"
print(q3)
response = chain.invoke(q3)
print(response.content)


Кто жил на острове, прежде чем туда приплыли герои книги?
Выполняется поиск...
..............................
На острове до прибытия героев жили трое: Сильвер, старый Морган и Бен — участники прежних злодеяний, связанных с сокровищами.


In [6]:
q4 = "Кого бросили несколько лет назад на острове, прежде чем туда приплыли герои книги?"
print(q4)
response = chain.invoke(q4)
print(response.content)


Кого бросили несколько лет назад на острове, прежде чем туда приплыли герои книги?
Выполняется поиск...
..............................
Сильвера, старого Моргана и Бена.


In [7]:
q5 = "Билли Боне как-то сказал что-то, и теперь он это проверяет на своей шкуре. Про что речь, про что он сказал?"
print(q5)
response = chain.invoke(q5)
print(response.content)


Билли Боне как-то сказал что-то, и теперь он это проверяет на своей шкуре. Про что речь, про что он сказал?
Выполняется поиск...
..............................
Про цвет крови.


In [8]:
# с агентом
search_tool = create_retriever_tool(
    retriever=retriever,
    name="search_knowledge_base",
    description="Ищет информацию в книге Остров Сокровищ"
)

agent = create_agent(
    model=llm,
    system_prompt="Ты полезный ассистент. Когда тебя спрашивают любую информацию, связанную с Островом сокровищ, пиратами, "
                  "используй инструмент search_knowledge_base для поиска информации,"
                  " и отвечай только с учетом полученного контекста. Если вопрос общего характера, отвечай сам, "
                  "без использования инструментов",
    tools=[search_tool]
)


In [9]:
# без поиска
response = agent.invoke({"messages": ["Как принято разговаривать на улицах городов?"]})
print(response['messages'][-1].content)


Принято разговаривать вежливо и уважительно, соблюдая правила этикета. Важно быть корректным в выражениях, избегать грубости и оскорблений. Также следует учитывать культурные особенности и традиции места, где происходит общение.


In [10]:
# с поискам
response = agent.invoke({"messages": ["С каким тоном пираты обычно разговаривали на Острове, приведи конкретные примеры."]})
print(response['messages'][-1].content)


Выполняется поиск...
..............................
Пираты на Острове сокровищ обычно разговаривали грубо и прямо, не стесняясь в выражениях. Вот несколько примеров:

1. **Грубый тон**: Пираты часто обращались друг к другу с использованием бранных слов и грубых выражений. Например, они могли окликать друг друга издали или обсуждать свои дела вполголоса, но при этом не стеснялись в выражениях.

2. **Прямота и требовательность**: В разговорах пираты часто были прямыми и требовательными. Например, один из них обращается к капитану Сильверу с просьбой о собрании, но при этом подчёркивает свои права и не стесняется в выражениях: *"— ну что ж, говорите, я слушаю. — прошу прощения, сэр, — начал один из пиратов. — вы часто нарушаете наши обычаи."*

3. **Угрозы и предупреждения**: Пираты могли использовать угрозы или предупреждения в разговорах. Например, они могли обсуждать свои планы и действия, не стесняясь в выражениях и подчёркивая свою силу и authority.

Таким образом, тон разговоров пира

In [ ]:
# Мне кажется, что лок ллм+раг плохо сработали. И хотелось бы потом узнать как повысить точность ответов.